In [294]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Bio import SeqIO
from BCBio import GFF
from Bio import SearchIO
import csv
import sqlite3
import time
from multiprocessing import Pool, cpu_count
import sys
from calc_icity_ import *
import ast
from collections import defaultdict
import subprocess
import logomaker


In [ ]:
# takes in any target p30 annotategff output (.gff file for geneious, .tsv file to summarize domains)
# merges these target p30 data with target_fetch_analysis.tsv files

### any sigma70 hit genome example
- 65 genomes
- goal: find trans acting transposases
- approach: search for pfam with "y1_tnp", "y2_tnp"
- filters: species with multiple tnpBs ("cas12f")


In [369]:
s70_genome_dir = "../annotate/annotategff/OUTPUT/sig70_genomes/"
s70_genome_tsv_paths =  [s70_genome_dir + file for file in os.listdir(s70_genome_dir) if ".tsv" in file]
genome_annot_dfs = []
for path in s70_genome_tsv_paths:
    genome_annot_df = pd.read_csv(path, sep = '\t')
    genome_annot_dfs += [genome_annot_df]
s70_allgenomes_annot_df = pd.concat(genome_annot_dfs).drop_duplicates()

/opt/conda/envs/ggdb/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [519]:
def filter_df_inclusive(df, column, keywords):
    keyword = keywords[0]
    filter_s = df[column] == keyword
    for keyword in keywords[1:]:
        filter_s = filter_s | (df[column] == keyword)
    print(sum(filter_s))
    return df[filter_s]
def filter_df_exclusive(df, column, keywords):
    keyword = keywords[0]
    filter_s = df[column].apply(lambda x: keyword in x if type(x) == str else False)
    for keyword in keywords[1:]:
        filter_s = filter_s & df[column].apply(lambda x: keyword in x if type(x) == str else False)
    print(sum(filter_s))
    return df[filter_s]
def filter_df_contains(df, column, keywords):
    keyword = keywords[0]
    filter_s = df[column].apply(lambda x: keyword in x if type(x) == str else False)
    for keyword in keywords[1:]:
        filter_s = filter_s | df[column].apply(lambda x: keyword in x if type(x) == str else False)
    print(sum(filter_s))
    return df[filter_s]

In [ ]:
bait_target_df = filter_df_contains(s70_allgenomes_annot_df, "attrib", ["Cas12f"])#, "Sigma70_r4"])
bait_target_df["seqname"].apply(lambda x: '_'.join(x.split('_')[:2])).value_counts()[:15]

0


Series([], Name: seqname, dtype: int64)

In [547]:
genome_manyloci_df = filter_df_contains(s70_allgenomes_annot_df, "seqname", ["GCF_004362395"])

19773


In [570]:
#filter_df_contains(genome_manyloci_df, "attrib", ["Y1_Tnp", "Y2_Tnp", "Cas12f", "OrfB", "IS91", "IS200", "IS1341"])

In [558]:
possible_loci_loc = list(filter_df_contains(s70_allgenomes_annot_df, "attrib", ["Cas12f"])["seqname"].unique())

291


In [559]:
s70_contigswloci_df = filter_df_contains(s70_allgenomes_annot_df, "seqname", possible_loci_loc)

519625


In [562]:
s70_loci_rows = filter_df_contains(s70_contigswloci_df, "attrib", ["Cas12f", "Sigma70_r4"])
s70_loci_rows.to_csv("sigma70_loci_genome.summary.csv")

2527


In [572]:
seqname_loci_filter = ['900113135', '002900095', '014653155', '002094855', '005049105', '003285545',
 '011064245', '014651535', '003285545', '011064245', '014651535', '004785645', '003259615', '013204795', '000747525', 
                       '013266735', '004362395', '000523515', '900115145', '000152985', '002736205', '000023465']
s70_multiloci_rows_df = filter_df_contains(s70_contigswloci_df, "seqname", seqname_loci_filter)

236532


In [575]:
filter_df_contains(s70_multiloci_rows_df, "attrib", ["Y1_Tnp", "Y2_Tnp", "IS91", "IS200"])

29


,seqname,source,feature,start,end,score,strand,frame,attrib,pfam_domain,pfam_eval,cct_domain,cct_eval,CDS_pid,CRISPR_start,CRISPR_end
54073,gnl|Bactopia|GCF_002900095_2,Prodigal:002006,CDS,596961,597392,.,+,0,ID=GCF_002900095_01125;Parent=GCF_002900095_01...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54075,gnl|Bactopia|GCF_002900095_2,hmmsearch,pfam_domain,596994,597348,1,+,0,ID=Y1_Tnp;domain_evalue=5e-39;domain_score=130...,Y1_Tnp,domain_evalue=5e-39,NaN,NaN,NaN,NaN,NaN
60999,gnl|Bactopia|GCF_002900095_2,Prodigal:002006,CDS,1072854,1073615,.,+,0,ID=GCF_002900095_01558;Parent=GCF_002900095_01...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61009,gnl|Bactopia|GCF_002900095_2,Prodigal:002006,CDS,1073612,1074736,.,+,0,ID=GCF_002900095_01559;Parent=GCF_002900095_01...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61011,gnl|Bactopia|GCF_002900095_2,hmmsearch,pfam_domain,1073672,1073915,1,+,0,ID=Zn_Tnp_IS91;domain_evalue=3e-27;domain_scor...,Zn_Tnp_IS91,domain_evalue=3e-27,NaN,NaN,NaN,NaN,NaN
61015,gnl|Bactopia|GCF_002900095_2,hmmsearch,pfam_domain,1074038,1074545,1,+,0,ID=Y2_Tnp;domain_evalue=3.8e-54;domain_score=1...,Y2_Tnp,domain_evalue=3.8e-54,NaN,NaN,NaN,NaN,NaN
61019,gnl|Bactopia|GCF_002900095_2,hmmsearch,pfam_domain,1074620,1074653,1,+,0,ID=Zn_Tnp_IS91;domain_evalue=3e-27;domain_scor...,Zn_Tnp_IS91,domain_evalue=3e-27,NaN,NaN,NaN,NaN,NaN
61042,gnl|Bactopia|GCF_002900095_2,Prodigal:002006,CDS,1077256,1078017,.,+,0,ID=GCF_002900095_01563;Parent=GCF_002900095_01...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61052,gnl|Bactopia|GCF_002900095_2,Prodigal:002006,CDS,1078014,1079138,.,+,0,ID=GCF_002900095_01564;Parent=GCF_002900095_01...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61053,gnl|Bactopia|GCF_002900095_2,hmmsearch,pfam_domain,1078074,1078317,1,+,0,ID=Zn_Tnp_IS91;domain_evalue=3e-27;domain_scor...,Zn_Tnp_IS91,domain_evalue=3e-27,NaN,NaN,NaN,NaN,NaN


In [585]:
is91_genomes = filter_df_contains(s70_allgenomes_annot_df, "attrib", ["IS91"]).reset_index()
is91_genomes["assembly_id"] = is91_genomes["seqname"].apply(lambda x: "_".join(x.split('|')[-1].split("_")[:2]))

163


In [ ]:
def get_annot_genome_fna(assembly_ids, outpath, num_bases_tnpb_orf):
    with open(outpath, "w") as outfile:
        for assembly_id in assembly_ids:
            gff_path = "../annotate/annotategff/OUTPUT/{}.sorted.gff".format(assembly_id)
            fna_path = "../annotate/annotategff/INPUT/fna/{}.examples.fna".format(assembly_id)
            with open(fna_path) as seq_handle:
                seq_dict = SeqIO.to_dict(SeqIO.parse(seq_handle, "fasta"))
            with open(gff_path) as in_handle:
                for rec in GFF.parse(in_handle):
                    for i in range(len(rec.features)):
                        if rec.features[i].type == "CDS_bait":
                            bait_loc = rec.features[i].location
                            bait_id = rec.features[i].id
                            rec_id = rec.id
                            dTnpB_loci_seq = seq_dict[rec_id].seq[bait_loc.end - num_bases_tnpb_orf:bait_loc.end + 250]
                            rec_bait_id = "{}|{}".format(rec_id.split('|')[0], bait_id)
                            print(">" + rec_bait_id, file=outfile)
                            print(dTnpB_loci_seq, file=outfile)

In [688]:
temp[temp["is91_fam"] == "ISWz1"].drop_duplicates().shape#temp[temp["is91_fam"] == "ISMno24"]#

(41, 20)

In [682]:
temp["is91_fam"] = temp["attrib"].apply(lambda x: x.split(';')[-2].split(" ")[-1])
temp[temp["is91_fam"] == "ISTha3"]

,level_0,index,seqname,source,feature,start,end,score,strand,frame,attrib,pfam_domain,pfam_eval,cct_domain,cct_eval,CDS_pid,CRISPR_start,CRISPR_end,assembly_id,is91_fam
18,35,60999,gnl|Bactopia|GCF_002900095_2,Prodigal:002006,CDS,1072854,1073615,.,+,0,ID=GCF_002900095_01558;Parent=GCF_002900095_01...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GCF_002900095,ISTha3
20,39,61042,gnl|Bactopia|GCF_002900095_2,Prodigal:002006,CDS,1077256,1078017,.,+,0,ID=GCF_002900095_01563;Parent=GCF_002900095_01...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GCF_002900095,ISTha3
22,43,22973,gnl|Bactopia|GCF_009829075_28,Prodigal:002006,CDS,12533,13312,.,+,0,ID=GCF_009829075_02132;Parent=GCF_009829075_02...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GCF_009829075,ISTha3
24,47,30501,gnl|Bactopia|GCF_014841125_13,Prodigal:002006,CDS,199,1014,.,-,0,ID=GCF_014841125_02305;Parent=GCF_014841125_02...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GCF_014841125,ISTha3
26,51,67937,gnl|Bactopia|GCF_014841125_9,Prodigal:002006,CDS,1461,2270,.,-,0,ID=GCF_014841125_01376;Parent=GCF_014841125_01...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GCF_014841125,ISTha3
28,55,12780,gnl|Bactopia|GCF_004785645_14,Prodigal:002006,CDS,9411,10250,.,-,0,ID=GCF_004785645_01636;Parent=GCF_004785645_01...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GCF_004785645,ISTha3
32,63,5144,gnl|Bactopia|GCF_016469035_102,Prodigal:002006,CDS,158465,159067,.,-,0,ID=GCF_016469035_04183;Parent=GCF_016469035_04...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GCF_016469035,ISTha3
34,67,9859,gnl|Bactopia|GCF_016469035_2,Prodigal:002006,CDS,193310,193912,.,-,0,ID=GCF_016469035_00363;Parent=GCF_016469035_00...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GCF_016469035,ISTha3
35,68,46651,gnl|Bactopia|GCF_016469035_57,Prodigal:002006,CDS,12117,12866,.,+,0,ID=GCF_016469035_02924;Parent=GCF_016469035_02...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GCF_016469035,ISTha3
37,72,47325,gnl|Bactopia|GCF_016469035_59,Prodigal:002006,CDS,4183,4785,.,-,0,ID=GCF_016469035_02981;Parent=GCF_016469035_02...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GCF_016469035,ISTha3


In [678]:
temp = is91_genomes[is91_genomes["source"] != "hmmsearch"].reset_index()
temp["assembly_id"].value_counts()

GCF_900142975    16
GCF_016469035    11
GCF_006974145     9
GCF_000733925     8
GCF_004362395     6
GCF_016428595     4
GCF_900115145     4
GCF_007827275     4
GCF_002900095     4
GCF_004785645     3
GCF_014841125     3
GCF_900113135     3
GCF_009829075     2
GCF_000152985     2
GCF_000687915     1
GCF_007785775     1
GCF_900489465     1
GCF_900106795     1
GCF_011030405     1
GCF_011043555     1
GCF_900116225     1
Name: assembly_id, dtype: int64

### dTnpB example, any residue mutated
- 28 target p30 clusters
- length filtered, junk baits removed

In [136]:
dtnpB_annotate_dir = "../annotate/annotategff/OUTPUT/dtnpbs_anymut_filtered/"
dtnpB_targetsummarystats = "../ggdb_dfs/dTnpB_anymut_target_analysis.filtered.tsv"
dtnpB_targets_df = pd.read_csv(dtnpB_targetsummarystats, sep = '\t')
dtnpB_targets = list(dtnpB_targets_df.loc[:,'target_p30id'])

In [214]:
dtnpB_allmut = "../ggdb_dfs/dTnpB_allmut_target_fetch.input.csv"
dtnpB_allmut_df = pd.read_csv(dtnpB_allmut)
dtnpB_allmut_targets = list(dtnpB_allmut_df.loc[:,'target_p30id'])

In [219]:
set(dtnpB_targets).symmetric_difference(dtnpB_allmut_targets).intersection(dtnpB_targets)

{'27177e034d397f4d56',
 '56b1b1c63eabb8641e',
 '57594ff4d26b78158a',
 '5f114e10fd531e6e37',
 'c7ce7035555cc371c6',
 'e5c44d78594897b19b'}

In [632]:
target_annot_dfs = []
for pid in dtnpB_targets:
    target_tsv_path = dtnpB_annotate_dir + "{}.final.tsv".format(pid)
    target_annot_df = pd.read_csv(target_tsv_path, sep = '\t')
    target_annot_dfs += [target_annot_df]

In [633]:
dtnpb_alltargetloci_stats_df[dtnpb_alltargetloci_stats_df["pfam_domain"] == "Sigma70_r4"]["target_p30id"].value_counts()

aa6af7e9289c3558d3    146
23422406293d40c201    108
c1050b21cc75640d51     80
6bf6c4c7da68779d7a     66
c7ce7035555cc371c6     12
49c65ceeadb463ef7d     10
d862301d73972ea41c      7
27177e034d397f4d56      6
3b91e97121e56aa047      4
e95a232b429ceb001b      4
7684dc2cf4ee624258      4
9038f7993863bcadca      4
0b921ace501a0fe980      3
260273e052f76dffd2      3
4dcbca21fe7e9a5c29      3
4e45823b6b83464d6e      2
57594ff4d26b78158a      2
56b1b1c63eabb8641e      2
04bbfb58c48a0e8a29      2
0a3217e07a25d0cbe0      1
Name: target_p30id, dtype: int64

In [638]:
dtnpb_alltargetloci_stats_df.shape

(117321, 28)

In [637]:
dtnpb_alltargetloci_stats_df = pd.concat(target_annot_dfs).drop_duplicates()
dtnpb_alltargetloci_stats_df['target_p100id'] = dtnpb_alltargetloci_stats_df['seqname'].apply(lambda x: x.split('|')[2])
dtnpb_alltargetloci_stats_df["len"] = np.abs(list(dtnpb_alltargetloci_stats_df["end"] - dtnpb_alltargetloci_stats_df["start"]))
dtnpb_target_stats_df = dtnpb_alltargetloci_stats_df[dtnpb_alltargetloci_stats_df["in_target"] == True]
dtnpb_bait_stats_df = dtnpb_alltargetloci_stats_df[dtnpb_alltargetloci_stats_df["in_bait"] == True]
dtnpb_target_stats_df[dtnpb_target_stats_df["pfam_domain"] == "Sigma70_r4"]["target_p30id"].value_counts()

aa6af7e9289c3558d3    134
23422406293d40c201    100
c1050b21cc75640d51     76
6bf6c4c7da68779d7a     60
Name: target_p30id, dtype: int64

In [278]:
dtnpB_tnpA_atloci = dtnpb_alltargetloci_stats_df[dtnpb_alltargetloci_stats_df['pfam_domain'] == "RVT_1"]
dtnpB_tnpA_atloci["target_p30id"].value_counts()

aa6af7e9289c3558d3    14
c7ce7035555cc371c6     9
Name: target_p30id, dtype: int64

In [661]:
#get dictionary form t100 to b100
dtnpB_target_features_df = dtnpb_alltargetloci_stats_df[dtnpb_alltargetloci_stats_df["feature"] == "CDS_target"].drop_duplicates()
dtnpB_t100_b100_df = dtnpB_target_features_df.loc[:, ["target_p30id", "bait_p100id", "target_p100id"]]
t100_b100_filter_s = (dtnpB_t100_b100_df["target_p30id"] == "aa6af7e9289c3558d3") | \
(dtnpB_t100_b100_df["target_p30id"] == "23422406293d40c201") | \
(dtnpB_t100_b100_df["target_p30id"] == "c1050b21cc75640d51") | \
(dtnpB_t100_b100_df["target_p30id"] == "6bf6c4c7da68779d7a")
dtnpB_t100_b100_df = dtnpB_t100_b100_df[t100_b100_filter_s].drop_duplicates()
dtnpB_t100_b100_df.to_csv("../ggdb_dfs/sig70_b100_t100.csv")

In [275]:
dtnpB_tnpA_inneighb = dtnpB_tnpA_atloci[dtnpB_tnpA_atloci["in_target"] == False]
dtnpB_tnpA_inneighb[dtnpB_tnpA_inneighb["target_p30id"] == 'aa6af7e9289c3558d3']

,Unnamed: 0,seqname,source,feature,start,end,score,strand,frame,attrib,...,bait_start,bait_end,in_target,in_bait,target_p30id,bait_p100id,bait_p30id,CRISPR_start,CRISPR_end,len
4524,4524,aa6af7e9289c3558d3|cefd95a47094d0edf2|692dc04f...,hmmsearch,pfam_domain,21494,21677,1,+,0,ID=Y1_Tnp;domain_evalue=4.9e-06;domain_score=2...,...,13299,14396,False,False,aa6af7e9289c3558d3,3df53e78aaeeb7b83a,14bc66ab7912db1e07,NaN,NaN,183
4525,4525,aa6af7e9289c3558d3|cefd95a47094d0edf2|692dc04f...,hmmsearch,pfam_domain,21494,21677,1,+,0,ID=Y1_Tnp;domain_evalue=4.9e-06;domain_score=2...,...,13299,14396,False,False,aa6af7e9289c3558d3,3df53e78aaeeb7b83a,14bc66ab7912db1e07,NaN,NaN,183


In [234]:
dtnpb_sig70_stats_df = dtnpb_target_stats_df[dtnpb_target_stats_df["pfam_domain"] == "Sigma70_r4_2"]
dtnpb_sig70_stats_df["target_p30id"].value_counts()

aa6af7e9289c3558d3    132
23422406293d40c201     86
c1050b21cc75640d51     76
6bf6c4c7da68779d7a     60
e5c44d78594897b19b      1
Name: target_p30id, dtype: int64

In [252]:
s70_metadata_s = pd.Series(dtnpb_sig70_stats_df["seqname"].value_counts().index).sort_values()
s70_metadata_s.str.split(pat = '|', expand=True).to_csv("tmp.out.csv")

In [ ]:
dtnpb_targets_df = dtnpb_alltargetloci_stats_df[dtnpb_alltargetloci_stats_df["feature"] == "CDS_target"]
plt.hist(dtnpb_targets_df["len"]/3)

In [225]:
get_prot_sequence("09060ada4cdc297479")

'MPVDLSEIEAAEVELEQWLVEQTENGVPEIVLVGLLRDYADDIEHLGYVPRMWGESEQ*'

In [194]:
len("MGEKRSNHTVYNVNYHFVWCPKYRPEILEPIEDSLETSFRDVCDEYGYDILSLHISSDHVHLFLSAHPKHAPSEIVRTVKSITAREMWEQHESFWRGICGVVGSERTCTTSGRQATFRQTRLSSILSARNTFSGAYGLHPSLYRRIYRS")

149

In [142]:
dtnpb_targets_df[dtnpb_targets_df["len"] > 1500]["target_p30id"].drop_duplicates()

65    214319fd8cc31efee9
39    c7a1b8f01c5b7c48a2
Name: target_p30id, dtype: object

In [181]:
sum(dtnpb_target_stats_df["pfam_domain"].apply(lambda x: "ArsB" in x if type(x) == str else False))

True

In [189]:
sum(dtnpb_bait_stats_df["pfam_domain"].apply(lambda x: "ArsB" in x if type(x) == str else False))

136

In [221]:
def filter_target_domains(target_dom, bad_domains):
    if type(target_dom) == str:
        for domain in bad_domains:
            if domain in target_dom:
                return False
        return True
    else:
        return True
bad_bait_domains = ["LPG", "ArsB", "SpoIIE", "Oxidored_FMN", "APH", "CitMHS"]
dtnpb_goodbait_stats_df = dtnpb_bait_stats_df[dtnpb_bait_stats_df["pfam_domain"].apply(lambda x: filter_target_domains(x, bad_bait_domains))]

In [224]:
set(dtnpB_targets).symmetric_difference(dtnpB_allmut_targets).intersection(dtnpB_targets)

{'27177e034d397f4d56',
 '56b1b1c63eabb8641e',
 '57594ff4d26b78158a',
 '5f114e10fd531e6e37',
 'c7ce7035555cc371c6',
 'e5c44d78594897b19b'}

In [ ]:
dtnpb_goodbait_stats_df.loc[:, ["target_p30id", "pfam_domain"]].value_counts()

In [301]:
fna_file = "../annotate/annotategff/INPUT/fna/{}.examples.fna".format('aa6af7e9289c3558d3')
seq_handle = open(fna_file)
seq_dict = SeqIO.to_dict(SeqIO.parse(seq_handle, "fasta"))

In [ ]:
# GFF parsing to find coords

In [357]:
def get_sequence_boundaries(bait_loc, target_loc):
    b_start, b_end, t_start, t_end = bait_loc.start, bait_loc.end, target_loc.start, target_loc.end
    if b_start < t_start:
        start = b_start - 300
        end = t_end + 300
    else:
        start = t_start - 300
        end = t_end + 300
    return (start, end)

In [ ]:
in_file = "../annotate/annotategff/OUTPUT/{}.sorted.gff".format('aa6af7e9289c3558d3')
in_handle = open(in_file)
for rec in GFF.parse(in_handle):
    for i in range(len(rec.features)):
        if rec.features[i].type == "CDS_target":
            bait_loc = rec.features[i].location
            # print(rec.id, rec.features[i].location,
            #      rec.features[i].id,
            #      rec.features[i].type)
        if rec.features[i].type == "CDS_bait":
            target_loc = rec.features[i].location
    if bait_loc and target_loc:    
        boundaries = get_sequence_boundaries(bait_loc, target_loc)
        print(bait_loc, target_loc, boundaries)
        print(rec.id)
        seq = seq_dict[rec.id].seq
    # write new gff
    bait_loc, target_loc = False, False
    asdf

In [326]:
a

43

# sigma factors

In [78]:
s70_annotate_dir = "../annotate/annotategff/OUTPUT/sig70_target_filtered/"
s70_targetsummarystats = "../ggdb_dfs/sig70_target_fetch.input.tsv"
s70_targets_df = pd.read_csv(s70_targetsummarystats, sep = '\t')
s70_targets = list(s70_targets_df.loc[:,'target_p30id'])

In [29]:
s70_target_annot_dfs = []
for pid in s70_targets:
    target_tsv_path = s70_annotate_dir + "{}.final.tsv".format(pid)
    target_annot_df = pd.read_csv(target_tsv_path, sep = '\t')
    s70_target_annot_dfs += [target_annot_df]


In [96]:
s70_alltargetloci_stats_df = pd.concat(s70_target_annot_dfs).drop_duplicates()
s70_alltargetloci_stats_df["len"] = np.abs(list(s70_alltargetloci_stats_df["end"] - s70_alltargetloci_stats_df["start"]))
s70_target_stats_df = s70_alltargetloci_stats_df[s70_alltargetloci_stats_df["in_target"] == True]
s70_bait_stats_df = s70_alltargetloci_stats_df[s70_alltargetloci_stats_df["in_bait"] == True]

In [ ]:
s70_targets_df = s70_alltargetloci_stats_df[s70_alltargetloci_stats_df["feature"] == "CDS_target"]
plt.hist(s70_targets_df["len"]/3)

In [98]:
s70_targets_df[s70_targets_df["len"] > 1500].drop_duplicates()

,seqname,source,feature,start,end,score,strand,frame,attrib,pfam_domain,...,bait_start,bait_end,in_target,in_bait,target_p30id,bait_p100id,bait_p30id,CRISPR_start,CRISPR_end,len
2928,ed4afa22b5e938b715|d438a2460104f4c91a|d438a246...,PRODIGAL,CDS_target,12500,14344,1,+,0,ID=d438a2460104f4c91a,NaN,...,8983,9777,False,False,ed4afa22b5e938b715,acddb38f14873225e8,c1050b21cc75640d51,22293.0,26844.0,1844
10836,ed4afa22b5e938b715|ed4afa22b5e938b715|ed4afa22...,PRODIGAL,CDS_target,8635,10419,1,+,0,ID=ed4afa22b5e938b715,NaN,...,5302,6096,False,False,ed4afa22b5e938b715,a95cff64ff7885510a,c1050b21cc75640d51,17092.0,17859.0,1784


In [101]:
def get_prot_sequence(pid):
    con = sqlite3.connect("80kprotein_stats.db")
    cur = con.cursor()
    cmd = "SELECT sequence FROM proteins WHERE pid = '%s'" % pid 
    #print(cmd)
    cur.execute(cmd)
    return str(cur.fetchone()[0])
    con.close()
get_prot_sequence('226611224045add90b')
get_prot_sequence('717024767c388b708a')

'MAKLEEIAELLTEEIYGFNKSIDRLEELSEYLGDIKVRADATNIEQLLQKHFHKQEENLSGQSSKLDKVYETMEDTYLLPRWIVILFFVCVGVTFLSVGAAIFYYQKNDNTKETMTQYIRHYNSFFKDTPEAKKLYQEWTDKEKNRAK*'

In [ ]:
s70_target_stats_df.loc[:, ["pfam_domain", "target_p30id"]].value_counts()

In [103]:
s70_alltargetloci_stats_df = pd.concat(s70_target_annot_dfs)
s70_target_stats_df = s70_alltargetloci_stats_df[s70_alltargetloci_stats_df["in_target"] == True]